In [1]:
#mount the right drive and authenticate
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from PIL import Image
import os
import gc
import matplotlib.pyplot as plt
import time
import datetime

os.chdir('/content/drive/My Drive/Seg_net/library/cut')
from cut_image import cut_image

from google.colab import output


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
############################### ~ PARAMETERS ~ #################################
LABELS = ["PASCOLO", "BOSCO", "NON COLTIVABILE"]

N_of_classes = len(LABELS) # Number of classes

N_of_band = 5 #imAge band 3 for RGB + 1 for NIR + 1 DTM

patch_size = 128

percentage_of_acceptable_bad_pixels = 0.1 # % of wrong pixel per patch

str_prc = '10_'

# 1: patchsize_step = patch_size/overlapping -> NO OVERLAPPING
# 2: patchsize_step = patch_size/2 -> 50% OF OVERLAPPING
# 3: patchsize_step = patch_size/4 -> 25% OF OVERLAPPING
overlapping = 4/3
overlap_name = '_25_overlap'

#If true consider in the dataset even all borders
border = True

###############################################################################


############################# ~ DATA FOLDER ~ #################################
start_date = time.asctime(time.localtime(time.time()))

origin = '/content/drive/My Drive/Seg_net/dataset/raw_dtm/'
destination = '/content/drive/My Drive/Seg_net/dataset/lists/' + 'Patch_size' + str(patch_size) + ' prc' + str_prc + overlap_name + ' ' + start_date + '/'

os.mkdir(destination)
os.mkdir(destination + 'data/')
os.mkdir(destination + 'tare/')

os.chdir(origin)

directories = os.listdir('.')

###############################################################################

########################### ~ SUPPORT VARIABLES ~ ############################
unique_value = np.array([0,0])

cycle_time = []

patch_data_list = np.zeros([0,patch_size,patch_size, N_of_band], dtype = np.float32)

patch_tare_list = np.zeros([0,patch_size,patch_size, N_of_classes], dtype = np.float32)

###############################################################################

############################## ~ DATA PROCESING ~ ##############################

i = 0


for cur_dir in directories:

    start_time = time.time()
    i = i + 1
    print('{} Loading {}'.format(i,cur_dir))
    os.chdir(origin + cur_dir)

    #Load data
    RGB = np.load(cur_dir + "_RGB.npy")
    NIR = np.load(cur_dir + "_NIR.npy")[:,:,0]
    DTM = np.load("dtm135.npy")
    #Allocate memory
    RGB_NIR = np.empty(shape=(RGB.shape[0],RGB.shape[1],N_of_band), dtype=np.float32)
    #fufill with data
    RGB_NIR[:,:,0:3] = RGB
    RGB_NIR[:,:,3] = NIR[0:NIR.shape[0],0:NIR.shape[1]]
    RGB_NIR[:,:,4] = DTM
    #Load tara
    tare = np.load("tare.npy")

    print("RGB_NIR shape {} of type {}".format(RGB_NIR.shape,RGB_NIR.dtype))
    print("Tare shape {} of type {}".format(tare.shape,tare.dtype))

    available_classes = np.unique(tare)

    #Save unique tare value fot this image
    unique_value = np.concatenate((unique_value, available_classes))


    map_600 = np.zeros((tare.shape), dtype = np.float32)
    map_700 = np.zeros((tare.shape), dtype = np.float32)
    map_800 = np.zeros((tare.shape), dtype = np.float32)

    if (638 in available_classes):
        map_600 = np.where(tare == 638, 1, map_600)
    if (659 in available_classes):
        map_600 = np.where(tare == 659, 1, map_600)
    if (654 in available_classes):
        map_600 = np.where(tare == 654, 1, map_600)

    if (656 in available_classes):
        map_700 = np.where(tare == 656, 1, map_700)
    if (650 in available_classes):
        map_700 = np.where(tare == 650, 1, map_700)
        
    if (660 in available_classes):
        map_800 = np.where(tare == 660, 1, map_800)    
    if (690 in available_classes):
        map_800 = np.where(tare == 690, 1, map_800)
    if (780 in available_classes):
        map_800 = np.where(tare == 780, 1, map_800)
    if (770 in available_classes):
        map_800 = np.where(tare == 770, 1, map_800)


    tare_splitted = np.zeros((tare.shape[0],tare.shape[1],N_of_classes), dtype = np.float32)

    tare_splitted[:,:,0] = map_600
    tare_splitted[:,:,1] = map_700
    tare_splitted[:,:,2] = map_800

    print("Tare shape is {}\n".format(tare_splitted.shape))

    temp1, temp2 = cut_image(RGB_NIR, tare, tare_splitted,patch_size,percentage_of_acceptable_bad_pixels,overlapping,border)

    patch_data_list = np.concatenate((patch_data_list, temp1))
    patch_tare_list = np.concatenate((patch_tare_list, temp2))

    print("Created:\nRGB_NIR patches {}\nTare {}".format(temp1.shape,temp2.shape))

    print("Actual size:\nRGB_NIR patches {}\nTare {}\n".format(patch_data_list.shape,patch_tare_list.shape))

    elapsed_time = time.time() - start_time
    cycle_time.append(elapsed_time)
    it_mean_duration = np.mean(cycle_time)

    print('Iter take {} in mean. Subsampling process running from {} of approximately {}'.format(it_mean_duration,
                                            time.strftime('%H h %M m %S s', time.gmtime(it_mean_duration * i)),
                                            time.strftime('%H h %M m %S s', time.gmtime(it_mean_duration * len(directories)))))

    del map_600, map_700, map_800, temp1, temp2, tare_splitted, RGB_NIR, tare, RGB, NIR
    gc.collect()

os.chdir(destination)
#Create a specification file
f= open("info.txt","w+")
f.write("patch_size = %d\r\n" % (patch_size))
f.write("percentage_of_acceptable_bad_pixels = %f\r\n" % (percentage_of_acceptable_bad_pixels))
f.write("overlapping = %f\r\n" % (overlapping))
f.write("border = %d\r\n" % (border))
f.write("N_of_band = %d\r\n" % (N_of_band))
f.write("N_of_classes = %d\r\n" % (N_of_classes))
f.write("Number of images processed = %d\r\n" % (len(directories)))
f.write("Number of patches = %d\r\n" % (patch_data_list.shape[0]))
f.close()

#Data_save
os.chdir(destination + 'data/')
np.save('data.npy', patch_data_list)
os.chdir(destination + 'tare/')
np.save('tare.npy', patch_tare_list)

print('Found Classes {}'.format(np.unique(unique_value)))

known_classes = np.array([0, 638, 650, 654, 656, 659, 660, 690, 770, 780])
print('Known Classes {}'.format(np.unique(known_classes)))
###############################################################################


1 Loading 025154w
RGB_NIR shape (3060, 3470, 5) of type float32
Tare shape (3060, 3470) of type int16
Tare shape is (3060, 3470, 3)

Created:
RGB_NIR patches (74, 128, 128, 5)
Tare (74, 128, 128, 3)
Actual size:
RGB_NIR patches (74, 128, 128, 5)
Tare (74, 128, 128, 3)

Iter take 4.054673671722412 in mean. Subsampling process running from 00 h 00 m 04 s of approximately 00 h 24 m 11 s
2 Loading 025152w
RGB_NIR shape (3060, 3470, 5) of type float32
Tare shape (3060, 3470) of type int16
Tare shape is (3060, 3470, 3)

Created:
RGB_NIR patches (138, 128, 128, 5)
Tare (138, 128, 128, 3)
Actual size:
RGB_NIR patches (212, 128, 128, 5)
Tare (212, 128, 128, 3)

Iter take 6.233865141868591 in mean. Subsampling process running from 00 h 00 m 12 s of approximately 00 h 37 m 11 s
3 Loading 025153w
RGB_NIR shape (3060, 3470, 5) of type float32
Tare shape (3060, 3470) of type int16
Tare shape is (3060, 3470, 3)

Created:
RGB_NIR patches (58, 128, 128, 5)
Tare (58, 128, 128, 3)
Actual size:
RGB_NIR pa

In [3]:

print('Known Classes {}'.format(np.unique(known_classes)))

Known Classes [  0 638 650 654 656 659 660 690 770 780]
